In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score as acc
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import math
from sklearn import model_selection
import os
import torch
from torch import nn
torch.manual_seed(3)
torch.cuda.manual_seed_all(3)

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# Defining modle metrics, will be used in the end
from sklearn.metrics import r2_score
def R2(y_true, y_pred):
    return r2_score(y_true, y_pred)
from sklearn.metrics import mean_squared_log_error
def MSLE(y_true, y_pred):
    return mean_squared_log_error(y_true, y_pred)
from sklearn.metrics import explained_variance_score
def EX_VAR(y_true, y_pred):
    return explained_variance_score(y_true, y_pred)
from sklearn.metrics import mean_absolute_error
def MAE(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)
from sklearn.metrics import mean_squared_error
def RMSE(y_true, y_pred):
    return mean_squared_error(y_true, y_pred,squared=True)
def MAP(y_true, y_pred):
    error=0
    for yt, yp in zip(y_true, y_pred):
        error += (yt - yp) / yt
    return (error / len(y_true))*100


In [4]:
#Reading the data and droping a useless column and droping all rows with Nan values in any column
#Scenario V-Modified.xlsx
DataSet = pd.read_excel( 'data.xlsx').drop(columns=['name','DOI','content (%)', 'concrete elastic modulus(GPa)',
                                                          'Ductility index(deltap/deltay)','steel fiber type' ]).dropna()

# adding a new column with name Kfold, later we will use it 
data = DataSet.copy()

# scaling
scaler = MinMaxScaler()
scaler.fit(data)
data = pd.DataFrame(scaler.transform(data.values), index=data.index, columns=data.columns)

data['kfold'] = -1
print(data.columns)
print(len(data))
data.head(5)

Index(['fiber aspect ratio(mm/mm)', 'fiber density(g/cm3)',
       'volume fraction(%)', 'reinforcement ratio(%)', 'beam width(mm)',
       'beam height(mm)', 'beam length(mm)', 'shear span-depth ratio',
       'fiber tensile strength(MPa)', 'Ductility index(deltau/deltay)',
       'Max Load', 'Deltap', 'kfold'],
      dtype='object')
184


D:\Software\Anaconda\envs\MachineLearning\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


,fiber aspect ratio(mm/mm),fiber density(g/cm3),volume fraction(%),reinforcement ratio(%),beam width(mm),beam height(mm),beam length(mm),shear span-depth ratio,fiber tensile strength(MPa),Ductility index(deltau/deltay),Max Load,Deltap,kfold
0,0.000,0.0,0.000000,0.132394,0.25,0.171429,0.571429,0.683333,0.000000,0.390661,0.058660,0.522806,-1
1,0.000,0.0,0.000000,0.211268,0.25,0.171429,0.571429,0.683333,0.000000,0.224125,0.098821,0.402627,-1
2,0.650,1.0,0.666667,0.132394,0.25,0.171429,0.571429,0.683333,0.927787,0.159533,0.086762,0.153214,-1
3,0.650,1.0,0.666667,0.211268,0.25,0.171429,0.571429,0.683333,0.927787,0.145525,0.128629,0.107881,-1
4,0.975,1.0,0.666667,0.132394,0.25,0.171429,0.571429,0.683333,0.831947,0.159144,0.093588,0.164952,-1


In [5]:
#Random shuffling the data 
df = data.sample(frac= 1, random_state=10).reset_index(drop=True)

#Adding numbers for each fold of data to the Kfold column 
kf = model_selection.KFold(n_splits=5)
for fold, (trn_, val_) in enumerate(kf.split(X=df)):
    df.loc[val_, 'kfold'] = fold

# check the Kfold column, now we can select a scpecific chunk of data with using Kfold column
df.head(3)

,fiber aspect ratio(mm/mm),fiber density(g/cm3),volume fraction(%),reinforcement ratio(%),beam width(mm),beam height(mm),beam length(mm),shear span-depth ratio,fiber tensile strength(MPa),Ductility index(deltau/deltay),Max Load,Deltap,kfold
0,0.00,0.000000,0.000000,0.126761,0.125,0.214286,0.714286,0.516667,0.407654,0.052529,0.086989,0.090833,0
1,0.45,0.987342,0.666667,0.098592,0.250,0.071429,0.200000,0.258333,0.366057,0.112840,0.076181,0.036333,0
2,0.64,0.987342,0.253333,0.021127,0.700,0.285714,0.771429,0.321667,0.447587,0.140078,0.032948,0.127446,0


In [6]:
# for ANN we use CPU of computer, so no need for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define Pytorch model
# This model has 3 layers. if you need more layer you should first define the layer in "__init__" function
# then use the layer in "forward" function

class TinyModel(torch.nn.Module):

    def __init__(self,n_inputs, l1, l2,l3, n_outputs):
        
        super(TinyModel, self).__init__()

        self.linear1 = torch.nn.Linear(n_inputs, l1)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(l1, l2)
        self.linear3 = torch.nn.Linear(l2, l3)
        self.linear4 = torch.nn.Linear(l3, n_outputs)
        #self.softmax = torch.nn.Softmax() use it if necessary (use it for classification task)

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.activation(x)
        x = self.linear3(x)
        x = self.activation(x)
        x = self.linear4(x)

        return x



Using cpu device


In [41]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class check_model(torch.nn.Module):

    def __init__(self,n_inputs, l1, l2,l3,l4,l5,l6, n_outputs):
        super(check_model, self).__init__()

        self.linear1 = torch.nn.Linear(n_inputs, l1)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(l1, l2)
        self.linear3 = torch.nn.Linear(l2, l3)
        self.linear4 = torch.nn.Linear(l3, l4)
        self.linear5 = torch.nn.Linear(l4, l5)
        self.linear6 = torch.nn.Linear(l5, l6)
        #self.linear7 = torch.nn.Linear(l6, l7)
        #self.linear8 = torch.nn.Linear(l7, l8)
        self.linear9 = torch.nn.Linear(l6, n_outputs)
        #self.softmax = torch.nn.Softmax() use it if necessary (for classification task)

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.activation(x)
        x = self.linear3(x)
        x = self.activation(x)
        x = self.linear4(x)
        x = self.activation(x)
        x = self.linear5(x)
        x = self.activation(x)
        x = self.linear6(x)
        x = self.activation(x)
        #x = self.linear7(x)
        #x = self.activation(x)
        #x = self.linear8(x)
        #x = self.activation(x)
        x = self.linear9(x)
        return x


Using cpu device


In [50]:
# hyperparameters for making a model

def creat_model():
    # check number of inputs and outputs    
    n_inputs = 9
    n_outputs = 3
    
    # build the model with given hyperparameters    
    model = check_model(n_inputs,64,128,256,512,512,1024,n_outputs)
    
    print(model)
    return model

def start_training():
    #Compiling the model
    epochs = 200
    model = creat_model()
    
    #chose the proper loss function
    #loss_fn = nn.L1Loss()
    loss_fn = nn.MSELoss()
    
    #chose the proper optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
    
    print(model)
    
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(x_train, y_train, model, loss_fn, optimizer)
        test(x_test, y_test, model, loss_fn)
    return model

In [51]:
def train(X,y, model, loss_fn, optimizer):
    # Change X and Y to Pytorch arrays
    X= torch.tensor(X.values, dtype = torch.float)
    y= torch.tensor(y.values, dtype = torch.float)
    model.train()

    # Compute prediction error
    pred = model(X)
    #y = torch.reshape(y,(list(y.size())[0],1))
    print(pred.shape)
    print(y.shape)
    loss = loss_fn(pred, y)
        # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [52]:
def test(X,y, model, loss_fn):
    X= torch.tensor(X.values, dtype = torch.float)
    y= torch.tensor(y.values, dtype = torch.float)
    with torch.no_grad():
        pred = model(X)
        print(R2(pred.numpy(),y.numpy()))
    return 

In [53]:
targets = ['Ductility index(deltau/deltay)','Max Load', 'Deltap','kfold']
inputs = [x for x in data.columns if x not in targets]
inputs.append('kfold')
print (inputs)

['fiber aspect ratio(mm/mm)', 'fiber density(g/cm3)', 'volume fraction(%)', 'reinforcement ratio(%)', 'beam width(mm)', 'beam height(mm)', 'beam length(mm)', 'shear span-depth ratio', 'fiber tensile strength(MPa)', 'kfold']


In [54]:
list_train, list_test = [] ,[]
list_R2_train, list_R2_test = [] ,[]
list_RMSE_train, list_RMSE_test = [],[]
list_MAE_train, list_MAE_test = [] ,[]
list_MAP_train, list_MAP_test =[] ,[]

# it is used for MinMax scale back to original values
#Mm =DataSet['OCR'].max() - DataSet['OCR'].min()

for i in range(5):
    # calling a fold and getting the X and Y for that fold
    test_fold = i
    x_all = df.drop(columns= targets)
    x_train = df[df['kfold']!= test_fold].drop(columns= targets)
    y_train = df[df['kfold']!= test_fold].drop(columns = inputs)
    x_test = df[df['kfold']== test_fold].drop(columns= targets)
    y_test = df[df['kfold']== test_fold].drop(columns = inputs)
    
    model = start_training()
#'''    
    # changing to Pytorch tensors for calculating model metrics
    x_train= torch.tensor(x_train.values, dtype = torch.float)
    x_test= torch.tensor(x_test.values, dtype = torch.float)
    x_all_tensor = torch.tensor(x_all.values, dtype = torch.float)
    
    # calling the model and making prediction
    y_pred_train = model(x_train).detach().numpy()
    y_pred_test  = model(x_test).detach().numpy()
    y_pred_all = model(x_all_tensor).detach().numpy()

    
    # saving the whol model output in each fold (becarefull about memory capacity)
    list_train.append((y_train,y_pred_train))
    list_test.append((y_test,y_pred_test))

    
    # saving the model metrics of each FOLD in a list
    list_R2_test.append(R2(y_test,y_pred_test))
    list_R2_train.append(R2(y_train,y_pred_train))
    
    list_RMSE_test.append(RMSE(y_test,y_pred_test))
    list_RMSE_train.append(RMSE(y_train,y_pred_train))
    
    list_MAE_test.append(MAE(y_test,y_pred_test))
    list_MAE_train.append(MAE(y_train,y_pred_train))
    

    #scale back if needed (only for single output)
    
    #y_pred_train = (y_pred_train * Mm) + DataSet['OCR'].min()
    #y_pred_test = (y_pred_test * Mm) + DataSet['OCR'].min()
    
    #y_train = (y_train * Mm) + DataSet['OCR'].min()
    #y_test = (y_test * Mm) + DataSet['OCR'].min()
    



check_model(
  (linear1): Linear(in_features=9, out_features=64, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=64, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=256, bias=True)
  (linear4): Linear(in_features=256, out_features=512, bias=True)
  (linear5): Linear(in_features=512, out_features=512, bias=True)
  (linear6): Linear(in_features=512, out_features=1024, bias=True)
  (linear9): Linear(in_features=1024, out_features=3, bias=True)
)
check_model(
  (linear1): Linear(in_features=9, out_features=64, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=64, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=256, bias=True)
  (linear4): Linear(in_features=256, out_features=512, bias=True)
  (linear5): Linear(in_features=512, out_features=512, bias=True)
  (linear6): Linear(in_features=512, out_features=1024, bias=True)
  (linear9): Linear(in_features=1024, out_features=3, bias=True)
)
Epoch 1


0.19843190521604384
Epoch 81
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.25688744605967395
Epoch 82
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.258791610643138
Epoch 83
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.2468865012210063
Epoch 84
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.287824747337216
Epoch 85
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.15706483024738216
Epoch 86
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.30986621143040355
Epoch 87
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.1973969006332057
Epoch 88
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.3193165287117218
Epoch 89
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.31536925439098346
Epoch 90
-------------------------------
torch.Size([147, 3]

0.5011811915245848
Epoch 170
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.474531120514823
Epoch 171
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.4553730656271182
Epoch 172
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.5123957302985147
Epoch 173
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.4220408106989659
Epoch 174
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.5058926200105089
Epoch 175
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.47846724989755424
Epoch 176
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.5006529580889749
Epoch 177
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.46822727052388163
Epoch 178
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.5334472385020282
Epoch 179
-------------------------------
torch.Size(

-1.126256294079182
Epoch 41
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-1.1840975215658818
Epoch 42
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.9349784001134784
Epoch 43
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.8097249472764112
Epoch 44
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.6595542432828154
Epoch 45
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.3573393012578783
Epoch 46
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.24388091996591732
Epoch 47
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.045512248956608735
Epoch 48
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.044335753302684945
Epoch 49
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.0394340737322851
Epoch 50
-------------------------------
torch.Siz

0.5557027590596134
Epoch 128
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.3889312803845198
Epoch 129
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.39782737880080926
Epoch 130
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.5222780161464436
Epoch 131
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.4885784572843746
Epoch 132
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.36424046228756707
Epoch 133
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.5165576999141177
Epoch 134
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.5439471142752219
Epoch 135
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.41558746377026584
Epoch 136
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.4508872682720777
Epoch 137
-------------------------------
torch.Siz

...

-1.3884060659989188
Epoch 58
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.6642098937054635
Epoch 59
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-2.038247203667588
Epoch 60
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-1.3465872695819208
Epoch 61
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.7606682097999581
Epoch 62
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-1.5501455820111552
Epoch 63
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-2.138891249182316
Epoch 64
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-1.2345327588141184
Epoch 65
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.7550166433513628
Epoch 66
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-1.1531206147217243
Epoch 67
-------------------------------
torch.Size([14

-0.2979948230131293
Epoch 144
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.11158254928207678
Epoch 145
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.028247719476646616
Epoch 146
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.20288441014300965
Epoch 147
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.10692755345018339
Epoch 148
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.016413282476759328
Epoch 149
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.20804046441289445
Epoch 150
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.06851292170544876
Epoch 151
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.12621737977210565
Epoch 152
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.10255760239366847
Epoch 153
------------------------

torch.Size([148, 3])
torch.Size([148, 3])
-112.0520539178923
Epoch 20
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-99.22486874906876
Epoch 21
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-73.49543393754723
Epoch 22
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-46.61295115654466
Epoch 23
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-30.062896711114547
Epoch 24
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-25.393549184933303
Epoch 25
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-24.177733819616353
Epoch 26
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-15.479538392539007
Epoch 27
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-10.12368890100784
Epoch 28
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-7.067930618344327
Epoch 29
--------

0.6536110079438795
Epoch 100
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.6577251472377944
Epoch 101
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.6789204550602079
Epoch 102
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.627336044831918
Epoch 103
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.6635873171081236
Epoch 104
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.6417477041626385
Epoch 105
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.653528771305702
Epoch 106
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.6809747115576285
Epoch 107
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.6474161364004501
Epoch 108
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.6766471865471494
Epoch 109
-------------------------------
torch.Size([14

0.7395248045768318
Epoch 188
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.757174239534225
Epoch 189
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.733078660813713
Epoch 190
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.7544335200388209
Epoch 191
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.7445148315621489
Epoch 192
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.7471000408926507
Epoch 193
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.7559059249646554
Epoch 194
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.7322878910091749
Epoch 195
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.7579581834934629
Epoch 196
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.7524467793267391
Epoch 197
-------------------------------
torch.Size([14

In [55]:
print('Neural Network')
print('  ')
print('%%% TEST %%%')
print('R2 = ',round(np.mean(list_R2_test),3))
print('RMSE = ',round(np.mean(list_RMSE_test),2))
print('MAE = ',round(np.mean(list_MAE_test),2))

#print(np.mean(list_R2_test))
print('  ')
print('*** TRAIN ***')
print('R2 = ',round(np.mean(list_R2_train),3))
print('RMSE = ',round(np.mean(list_RMSE_train),2))
print('MAE = ',round(np.mean(list_MAE_train),2))

#print(np.mean(list_R2_test))

Neural Network
  
%%% TEST %%%
R2 =  0.59
RMSE =  0.01
MAE =  0.06
  
*** TRAIN ***
R2 =  0.925
RMSE =  0.0
MAE =  0.02


In [19]:
#y_pred_all[:,0]
y_test = df[df['kfold']== test_fold].drop(columns = inputs)
df['ductility_index_prediction'] = y_pred_all[:,0]
df['max_load_prediction'] = y_pred_all[:,1]
df['delta p prediction'] = y_pred_all[:,2]

In [20]:
df.to_excel('MO_ANN_output.xlsx')